In [1]:
import pandas as pd
import json
from justwatch import JustWatch
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import get_from_table
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [3]:
film_id = 'f_02s8K'

In [4]:
with open('../data_prep/my_streaming_services.json', 'r') as schema:
    my_streaming_services = json.load(schema)
my_streaming_services_abbr = [x for x in set([x['provider_abbreviation'] for x in my_streaming_services]) if len(x) > 0]
abbr_to_full_dict = {x['provider_abbreviation']:x['streaming_service'] for x in my_streaming_services if len(x['provider_abbreviation']) > 0}
abbr_to_full_dict

{'amp': 'Amazon Prime Video',
 'nfx': 'Netflix',
 'bbc': 'BBC iPlayer',
 'al4': 'All4',
 'ntc': 'NowTV',
 'dnp': 'Disney+',
 'asd': 'Shudder',
 'ast': 'Lionsgate+',
 'plx': 'Plex',
 'ptv': 'PlutoTV',
 'myf': 'My5',
 'aim': 'Freevee',
 'itv': 'ITVX'}

In [5]:
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
film_url_title = get_from_table('FILM_TITLE', film_id, 'FILM_URL_TITLE')
if film_url_title == '':
    film_url_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
film_release_year = get_from_table('FILM_YEAR', film_id, 'FILM_YEAR')
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
print('Film ID is "{}", film title is "{}", film_url_title is "{}", film release year is "{}" and url is {}'.format(film_id, film_title, film_url_title, film_release_year, letterboxd_url))

Film ID is "f_02s8K", film title is "Silent Night, Deadly Night 5: The Toy Maker", film_url_title is "Silent Night, Deadly Night 5: The Toy Maker", film release year is "1991" and url is https://boxd.it/2s8K


In [6]:
just_watch = JustWatch(country='GB')
results = just_watch.search_for_item(query=film_url_title, release_year_from=film_release_year-1, release_year_until=film_release_year+1, page_size=1)
results

{'page': 1,
 'page_size': 1,
 'total_pages': 1,
 'total_results': 1,
 'items': [{'jw_entity_id': 'tm9214',
   'id': 9214,
   'title': 'Silent Night, Deadly Night 5: The Toy Maker',
   'full_path': '/uk/movie/silent-night-deadly-night-5-the-toy-maker',
   'full_paths': {'MOVIE_DETAIL_OVERVIEW': '/uk/movie/silent-night-deadly-night-5-the-toy-maker'},
   'poster': '/poster/138233299/{profile}',
   'poster_blur_hash': 'TCC}%?,-n2r;xbEg149^0}x^R%w[',
   'original_release_year': 1991,
   'object_type': 'movie',
   'offers': [{'jw_entity_id': 'tm9214',
     'monetization_type': 'free',
     'provider_id': 538,
     'package_short_name': 'plx',
     'currency': 'GBP',
     'urls': {'standard_web': 'https://watch.plex.tv/movie/silent-night-deadly-night-5-the-toy-maker?autoplay=1&utm_content=60bfff37c63765002dbaea87&utm_medium=deeplink&utm_source=justWatch-catalog',
      'deeplink_android_tv': 'intent://watch.plex.tv/movie/silent-night-deadly-night-5-the-toy-maker?autoplay=1&utm_content=60bfff3

In [9]:
first_result = results['items'][0]

y


In [18]:
provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
film_streaming_services_df['FILM_ID'] = film_id
film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
min_rental_price = min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])
min_rental_price

ValueError: min() arg is an empty sequence

In [20]:
min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])

ValueError: min() arg is an empty sequence

In [17]:

# delete_records('FILM_STREAMING_SERVICES', film_id)
if len(results['items']) > 0:
    first_result = results['items'][0]
    if first_result.get('title') == get_from_table('FILM_TITLE', film_id, 'FILM_TITLE'):
        provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
        valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
        if len(valid_abbr) > 0:
            valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
            film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
            film_streaming_services_df['FILM_ID'] = film_id
            film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
            film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
            # film_streaming_services_df['CREATED_AT'] = datetime.now()
            # df_to_table(film_streaming_services_df, 'FILM_STREAMING_SERVICES', replace_append='append', verbose=False)

In [39]:
min_rental_price = min([x['retail_price'] for x in first_result['offers'] if x['monetization_type'] == 'rent' and x['presentation_type'] == 'hd'])

In [23]:
[x for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]

[{'jw_entity_id': 'tm35196',
  'monetization_type': 'flatrate',
  'provider_id': 194,
  'package_short_name': 'ast',
  'currency': 'GBP',
  'urls': {'standard_web': 'https://watch.amazon.co.uk/detail?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d',
   'deeplink_web': 'https://watch.amazon.co.uk/watch?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d',
   'deeplink_android_tv': 'intent://watch.amazon.co.uk/watch?asin=B00L2OBRT6&time=0&territory=GB#Intent;package=com.amazon.amazonvideo.livingroom;scheme=https;end',
   'deeplink_fire_tv': 'intent://com.amazon.tv.launcher/detail?provider=aiv&providerId=B00L2OBRT6#Intent;package=com.amazon.tv.launcher;scheme=amzn;end',
   'deeplink_tvos': 'aiv://aiv/watch?gti=amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d&territory=GB&time=auto',
   'deeplink_tizenos': '{"id":"org.tizen.primevideo","action_data":"amzn1.dv.gti.b2a9f67f-683c-b79b-091e-66476824f38d"}',
   'deeplink_webos': '{ "id": "amazon", "params": { "contentTarget": "amzn1.dv